*2021/01/12 Tue*

<hr>

# 9-3. 템플릿 메타 프로그래밍(template metaprogramming)

템플릿을 통해서 타입이 마치 인자인 것처럼 사용하는 것을 바로 <u>일반화 프로그래밍(generic programming)</u>이라고 함.

In [1]:
/* 나만의 std::array 구현 */
#include <iostream>

template <typename T, unsigned int N>
class Array {
    T data[N];

public:
    // 배열에 대한 레퍼런스 arr
    Array(T (&arr)[N]) {
        for (int i = 0; i < N; i++) {
            data[i] = arr[i];
        }
    }
    
    T *get_array() { return data; }
    
    unsigned int size() { return N; }
    
    void print_all() {
        for (int i = 0; i < N; i++) {
            std::cout << data[i] << ", ";
        }
        std::cout << std::endl;
    }
};

int main() {
    int arr[3] = {1, 2, 3};
    
    // 배열 wrapper 클래스
    Array<int, 3> arr_w(arr);
    
    arr_w.print_all();
}

템플릿으로 생성한 어떤 두 타입이 같은지는 아래와 같은 코드로 확인할 수 있음.

`(typeid(Array<int, 3>) == typeid(Array(<int, 5>))` -> `0`

즉, 두 타입은 다름. 서로 다른 템플릿 인자로 인스턴스화되었으므로.

`typeid`를 사용할 때는 `<typeinfo>` 헤더 파일을 추가하면 된다.

In [1]:
template <int N>
struct Int {
    static const int num = N;
    /* 왜 static const에 값을 저장하냐면,
    첫 번째로, C++ 클래스 멤버 중에서, 
    클래스 자체에서 저런 식으로 초기화를 할 수 있는 멤버의 타입은 static const 밖에 없고,
    두번째로, static const야 말로 이 클래스는 이것이다라는 의미를 가장 잘 나타내기 때문.
    */
};

// 위와 같이 하면, 아래처럼 객체를 생성하듯 타입들을 생성할 수 있다.

typedef Int<1> one;
typedef Int<2> two;

In [ ]:
#include <iostream>
#include <typeinfo>

template <int N>
struct Int {
    static const int num = N;
};

////////
template <typename T, typename U>
struct add {
    typedef Int<T::num + U::num> result;  //
};
////////

int main() {
    typedef Int<1> one;
    typedef Int<2> two;
    
    typedef add<one, two>::result three;  // 실제 덧셈을 수행하는 부분
    
    std::cout << "Addition result : " << three::num << std::endl;
    // 3이 출력
}

`one`, `two`는 객체가 아니라 1, 2의 값을 나타내는 타입이 된다.
그런데, `one`, `two`를 가지고 재미있게도 마치 `int` 변수를 다루는 것처럼 연산자를 만들 수 있다!

`typedef add<one, two>::result three;`

이 부분이 실제 덧셈을 수행하는 부분. `add` 클래스를 함수라 생각한다면, 그 계산 결과를 내부 `result` 타입으로 반환한다고 보면 된다. `one`, `two`를 더한 것을 나타내는 타입이 `result`로 정의되고, 이를 `three`라 부른다는 것!

흥미로운 것은, 3이라는 값이 런타임 시에 계산되는 것이 아니라는 것! 컴파일 타임에 3으로 덧셈이 수행되고 치환되는 것!

## 템플릿 메타 프로그래밍(Template Meta Programming - TMP)

이 때까지 타입은 어떠한 객체에 무엇을 저장하느냐를 지정하는 데 사용해 왔지, 타입 자체가 어떠한 값을 가리키지는 않았다. 그러나, 템플릿을 사용하면 객체를 생성하지 않더라도 타입에 어떠한 값을 부여할 수 있고, 또 그 타입을 가지고 연산을 할 수 있다는 것을 이전 예제에서 다루었다.

타입은 반드시 컴파일 타임에 확정되어야 하므로, 컴파일 타임에 모든 연산이 끝난다. 이렇게 타입을 가지고 컴파일 타임에 생성되는 코드로 프로그래밍을 하는 것을 <u>메타 프로그래밍</u>이라 한다. C++의 경우 템플릿을 가지고 이러한 작업을 하기 때문에 템플릿 메타 프로그래밍, TMP라고 한다.

In [ ]:
#include <iostream>

template <int N>
struct Factorial {
    static const int result = N * Factorial<N - 1>::result;
};

// 템플릿 특수화
template <>
struct Factorial<1> {
    static const int result = 1;
};

int main() {
    std::cout << "6! = 1*2*3*4*5*6 = " << Factorial<6>::result << std::endl;
}

/*
int factorial(int n) {
    if (n == 1) return 1;
    
    return n * factorial(n - 1);
}
*/

`for` 문으로 구현할 수 있는 모든 코드는 똑같이 템플릿 메타 프로그래밍을 이용해서 구현할 수 있다.
그리고 `if` 문 역시 템플릿 특수화를 통해 TMP로 구현할 수 있다.

## TMP를 왜 쓰는가?

재미있는 사실은, 어떠한 C++ 코드도 템플릿 메타 프로그래밍 코드로 변환할 수 잇다는 것.(코드는 그 대신 굉장히 길어지겠지만,) 이렇게 하면 프로그램 실행 속도를 향상시킬 수 있다는 장점이 있음.(당연히 컴파일 시간은 엄청 늘겠지만,)

단, 템플릿 메타 프로그래밍으로 프로그램 전체를 구현하는 일은 없다. TMP는 매우 복잡하고, 컴파일 타임에 연산하는 것이므로 디버깅이 불가능하고, 템플릿 오류 시에 엄청난 길이의 오류를 내뿜는다.

그래서 TMP를 이용하는 경우는 꽤 제한적이지만, 많은 C++ 라이브러리들이 TMP를 이용하여 구현되었고(e.g., Boost 라이브러리), TMP를 통해서 컴파일 타임에 여러 오류들을 잡아낼 수 있고, 속도가 매우 중요한 프로그램의 경우 TMP를 통해서 런타임 속도도 향상시킬 수 있다.

In [ ]:
int gcd(int a, int b) {
    if (b == 0)
        return a;
    return gcd(b, a % b);
}

* 템플릿

In [ ]:
#include <iostream>

template <int X, int Y>
struct GCD {
    static const int value = GCD<Y, X % Y>::value;
};

template <int X>
struct GCD<X, 0> {
    static const int value = X;
};

int main() {
    std::cout << "gcd (36, 24) :: " << GCD<36, 24>::value << std::endl;
}

In [ ]:
#include <iostream>
#include <typeinfo>


template <int X, int Y>
struct GCD {
    static const int value = GCD<Y, X % Y>::value;
};
template <int X>
struct GCD<X, 0> {
    static const int value = X;
};

template <int N, int D = 1>
struct Ratio {
    typedef Ratio<N, D> type;
    // typedef로 자기 자신을 가리키는 타입을 넣어 줌. 마치 클래스에서의 this와 비슷.
    static const int num = N;  // 분자
    static const int den = D;  // 분모
};
template <class R1, class R2>
struct _Ratio_add {
    typedef Ratio<R1::num * R2::den + R2::num * R1::den, R1::den * R2::den> type;
};

template <class R1, class R2>
struct Ratio_add : _Ratio_add<R1, R2>::type {};  // 상속

int main() {
    typedef Ratio<2, 3> rat;  // 타입
    typedef Ratio<3, 2> rat2;  // 타입
    
    typedef Ratio_add<rat, rat2> rat3;
    // using rat3 = Ratio_add<rat, rat2>;로도 쓸 수 있고, 이게 typedef보다 이해하기 쉬움.

    std::cout << rat3::num << " / " << rat3::den << std::endl;

    return 0;
}

C++11부터 `typedef Ratio_add<rat, rat2> rat3;` 대신에 좀 더 직관적으로 `using rat3 = Ratio_add<rat, rat2>;`로 쓸 수 있음.

마찬가지로 `typedef void (*func)(int, int);` 대신 `using func = void (*)(int, int);`로 매우 직관적이게 나타낼 수 있음.

*-> typedef, using은 컴파일 타임에 타입을 치환하는 것과 연관된다고 보면 되겠네.*

* 총 코드

In [ ]:
#include <iostream>


template <int X, int Y>
struct GCD {
    static const int value = GCD<Y, X % Y>::value;
};

template <int X>
struct GCD {
    static const int value = X;
};

template <int X, int D = 1>
struct Ratio {
private:
    const static int _gcd = GCD<N, D>::value;
    
public:
    typedef Ratio<N / _gcd, D / _gcd> type;
    static const int num = N / _gcd;
    static const int den = D / _gcd;
};


template <class R1, class R2>
struct _Ratio_add {
    using type = Ratio<R1::num * R2::den + R2::num * R1::den, R1::den * R2::den>;
};
template <class R1, class R2>
struct Ratio_add : _Ratio_add<R1, R2>::type {};


template <class R1, class R2>
struct _Ratio_subtract {
    using type = Ratio<R1::num * R2::den - R2::num * R1::den, R1::den * R2::den>;
};
template <class R1, class R2>
struct Ratio_subtract : _Ratio_subtract<R1, R2>::type {};


template <class R1, class R2>
struct _Ratio_multiply {
    using type = Ratio<R1::num * R2::num, R1::den * R2::den>;
};
template <class R1, class R2>
struct Ratio_multiply : _Ratio_multiply<R1, R2>::type {};


template <class R1, class R2>
struct _Ratio_divide {
    using type = Ratio<R1::num * R2::den, R1::den * R2::num>;
};
template <class R1, class R2>
struct Ratio_divide : _Ratio_divide<R1, R2>::type {};


int main() {
    using r1 = Ratio<2, 3>;
    using r2 = Ratio<3, 2>;
    
    using r3 = Ratio_add<r1, r2>;
    std::cout << "2/3 + 3/2 = " << r3::num << " / " << r3::den << std::endl;
    
    using r4 = Ratio_multiply<r1, r3>;
    std::cout << "13/6 * 2/3 = " << r4::num << " / " << r4::den << std::endl;
}

## 생각 해보기

#### 문제 1
N 번째 피보나치 수를 나타내는 TMP 를 만들어보세요. 참고로 피보나치 수는, N 번째 항이 N - 1 번째 항과 N - 2 번째 항의 합으로 정의되는 수 입니다. 참고로 1, 1, 2, 3, 5, ... 로 진행됩니다.(난이도 : 하)

In [ ]:
#include <iostream>

template <int N>
struct fib {
    static const int result = fib<N - 1>::result + fib<N - 2>::result;
};
template <>
struct fib<1> { 
    static const int result = 1;
};
template <>
struct fib<2> {
    static const int result = 1;
};

int main() {
    std::cout << "5 번째 피보나치 수 :: " << fib<5>::result << std::endl;  // 5
}

#### 문제 2

TMP 를 사용해서 어떤 수가 소수인지 아닌지를 판별하는 프로그램을 만들어보세요. (난이도 : 상)참고로 이 문제는 다음 강좌에서 다룰 예정입니다!

In [ ]:
int main() {
    std::cout << boolalpha;
    std::cout << "Is prime ? :: " << is_prime<2>::result << std::endl;  // true
    std::cout << "Is prime ? :: " << is_prime<10>::result << std::endl;  // false
    
    std::cout << "Is prime ? :: " << is_prime<11>::result << std::endl;  // true
    std::cout << "Is prime ? :: " << is_prime<61>::result << std::endl;  // true
}

<hr>

# 9-4. 템플릿 메타 프로그래밍 2 

In [ ]:
bool is_prime(int N) {
    if (N == 2) return true;
    if (N == 3) return true;
    
    for (int i = 2; i <= N / 2; i++) {
        if (!(N % i)) return false;
    }
    
    return true;
}

In [ ]:
// 위의 코드를 템플릿 코드로 바꾼 것.

template <>
struct is_prime<2> {
    static const bool result = true;
};
template <>
struct is_prime<3> {
    static const bool result = true;
};
template <int N>
struct is_prime {
    static const bool result = !check_div<N, 2>::result;
};

template <int N, int d>
struct check_div {
    static const bool result = (N % d == 0) || check_div<N, d + 1>::result;
};
template <int N>
struct check_div<N, N / 2> {
    static const bool result = (N % (N / 2) == 0);
};

잘 짜여진 코드 같지만 컴파일 오류!

```
check_div<N,N/> : non-type parameter of a partial specialization must be a simple identifier
```

`<N, N / 2>` 부분이 문제인 것.

템플릿 부분 특수화 시에 반드시 다른 연산자가 붙지 않고 단순한 식별자만 입력해 주어야 한다.

`N`을 `int` 인자로 나타내는 대신에 아예 `N`을 나타내는 '타입'으로 구현하면 해결할 수 있지 않을까? 그러면 `N / 2` 역시 직접 계산하는 것이 아니라 `N / 2`을 나타내는 타입으로 대체할 수 있고 템플릿 부분 특수화 문제를 해결할 수 있다.

In [ ]:
template <int N>
struct INT {
    static const int num = N;  
};
template <typename a, typename b>
struct add {
    typedef INT<a::num + b::num>
};
template <typename a, typename b>
struct divide {
    typedef INT<a::num / b::num> result;
};

using one = INT<1>;
using two = INT<2>;
using three = INT<3>;
